In [1]:
from hearing_assistive import *

import numpy as np
%matplotlib inline

# Import Waves and Assign Targets

In [2]:
fire_alarm = read_wave('Sounds/school_fire_alarm.wav')
fire_alarm_target = 3

# Extract MFCC Features, Create Training Dataset, and Train SVM

In [3]:
# partition wave into segments ( default is 2.4 seconds )
seg_map = fire_alarm.partition()

In [4]:
from sklearn import linear_model

# initialize SVM classifier
clf = linear_model.SGDClassifier()   # linear SVM classifier with Stochastic Gradient Descent training

In [5]:
# TODO: MAKE THIS CELL A MODULE FUNCTION
# create and train dummyset
dataset = []
targetset = []
size = 100

# add (e.g. 100) dummy sets of zeros to the dataset
for i in range(size):
    dummyset = np.zeros((119, 13))
    dataset.append(dummyset)
    targetset.append(0)
    
# add (e.g. 100) dummy sets of ones to the dataset
for i in range(size):
    dummyset = np.ones((119, 13))
    dataset.append(dummyset)
    targetset.append(1)
    
# add (e.g. 100) dummy sets of negative ones to the dataset
for i in range(size):
    dummyset = np.negative(np.ones((119, 13)))
    dataset.append(dummyset)
    targetset.append(2)
    
# convert to numpy arrays
dataset = np.asarray(dataset)
targetset = np.asarray(targetset)
    
# restructure dataset before being processed by svm
dataset = dataset.reshape((dataset.shape[0], -1))

print len(dataset)
print dataset.shape
print len(targetset)
print targetset.shape
    
# train dummyset
clf.fit(dataset, targetset)

300
(300, 1547)
300
(300,)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [6]:
# create a dataset for each 2.4s-segment and feed each dataset into the SVM classifier
for i in range(len(seg_map)):
    # initialize dataset and targetset
    dataset = []
    targetset = []
    
    # get segment from seg_map
    segment = seg_map[i]
    
    # generate samples with added noise ( default is 100 samples ) and extract mfccs
    dataset, targetset = generate_feature_set(segment, fire_alarm_target)
    print i, dataset.shape, targetset.shape
    
    # restructure dataset before being processed by svm
    dataset = dataset.reshape((dataset.shape[0], -1))
    
    # fit classifier with dataset and corresponding targetset
    clf.partial_fit(dataset, targetset)

0 (100, 119, 13) (100,)
1 (100, 119, 13) (100,)
2 (100, 119, 13) (100,)
3 (100, 119, 13) (100,)
4 (100, 119, 13) (100,)
5 (100, 119, 13) (100,)
6 (100, 119, 13) (100,)
7 (100, 119, 13) (100,)
8 (100, 119, 13) (100,)
9 (100, 119, 13) (100,)
10 (100, 119, 13) (100,)
11 (100, 119, 13) (100,)
12 (100, 119, 13) (100,)
13 (100, 119, 13) (100,)
14 (100, 119, 13) (100,)
15 (100, 119, 13) (100,)
16 (100, 119, 13) (100,)
17 (100, 119, 13) (100,)
18 (100, 119, 13) (100,)
19 (100, 119, 13) (100,)
20 (100, 119, 13) (100,)
21 (100, 119, 13) (100,)


# Create Testing Dataset and Test SVM

In [11]:
testset = []
size = 2

# add dummy sets of zeros to the testset
for i in range(size):
    dummyset = np.zeros((119, 13))
    testset.append(dummyset)
    
# add dummy sets of ones to the testset
for i in range(size):
    dummyset = np.ones((119, 13))
    testset.append(dummyset)
    
# add dummy sets of negative ones to the testset
for i in range(size):
    dummyset = np.negative(np.ones((119, 13)))
    testset.append(dummyset)
    
# # add alarm sound files to the testset
# for i in range(size):
#     segment = seg_map[10]
#     spectrogram = segment.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
#     dummyset = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
#     testset.append(dummyset)
    
    
# convert to numpy array
testset = np.asarray(testset)
    
# restructure testset before being processed by svm
testset = testset.reshape((testset.shape[0], -1))
print testset.shape

(6, 1547)


In [12]:
print "extected: [0, 0, 1, 1, 2, 2, 3, 3]"
clf.predict(testset)

extected: [0, 0, 1, 1, 2, 2, 3, 3]


array([0, 0, 2, 2, 0, 0])